# Denver Crime Heatmap Notebook
CS 3120 Machine Learning Term Project - Konstantin Zaremski

---

This comprehensive notebook covers my project from ideation and planning, to exploratory data analysis (EDA), and final implementation.
### *********** Unfinished Draft (for peer review)

> ### Project Proposal
> ##### Introduction & Description
> After my car was vandalized in Denver earlier this October, I began ruminating on what makes an area “safe”. Eventually I came upon the Denver Public Crime Map which shows the 1000 most recent incidents on a map. This got me interested in the possibility of using this data in a predictive capacity. I am going to build a Denver crime prediction or likelihood map app, that will predict the likelihood of crime overall, predict the most likely crimes to be committed for different areas, and then present this as a heatmap visualization to the user.
> 
> ##### Dataset
> The source dataset for the public crime map is provided by Denver and contains 394,475 data points sourced from the FBI’s NIBRS database. After stripping off useless columns such as database keys and precinct numbers, the usable columns are the date and time of occurrence, type and category of crime, and longitude and latitude. Crimes do not happen because of a longitude and latitude value, so I am enriching the original dataset by adding additional features by querying each data point against Open Street Map data for land use, proximity to residential or main roads, building types, and amenities such as ATMs or RTD stations.
> 
> ##### Model
> I will be training two models to make predictions for my app: one model to enable the prediction of the total amount of crimes in an area given input features, and another model to perform multiclass classification to predict what type of crime is most likely to be committed in an area given input features. Since the input data and resulting crime or number of crimes is known and labeled, supervised learning models are best suited for this data. I will be using SciKit Learn’s HistGradientBoostingClassifier to predict crime type and HistGradientBoostingRegressor to predict crime volume. I am electing to use gradient boosting over decision trees or other models since it has a lower bias and should be more sensitive to different crime patterns.
> 
> ##### App Functionality
> The web app will be a simple map interface with a color heatmap overlay based on the underlying model’s crime predictions. The user will have options for a time delta, where they can view the heatmap/predictions for the current time plus or minus up to 48 hours. The user will have a way to toggle between seeing a multi-color heatmap for the types of crimes that are predicted, a single-color heatmap for the likelihood or predicted frequency of crime, and a heatmap that combines both, with areas having a lower predicted likelihood or frequency of crime being denoted by a more transparent version of the color representing the crime type for that area.
> 
> ##### Technical Implementation
> Since the app does not have user accounts, the backend can be implemented as a simple Flask app with an API endpoint to retrieve the heatmap overlay based on the map area the client says the user is currently viewing. Predictions will be stored in an SQLite database where they can be quickly retrieved to generate a heatmap at view time. The frontend will be implemented using simple HTML, CSS, and vanilla JavaScript to enable interactions. The map itself will be rendered using the Leaflet JavaScript map library/project.
> 
> ##### Feasibility
> Since the app itself is a visualization with only a few buttons/interactions, and no user accounts or user data collection, it will be simple to implement, which will allow for more time to be spent on tuning the models and finding the optimal way to slice the map or bin data.
> 

## Exploratory Data Analysis

### Setup
Importing required modules and libraries for EDA, etc. See `requirements_eda.txt` for a full list of requirements to install into the environment using `pip`.

In [3]:
from IPython.display import display
import pandas as pd
import math
import pyproj
import numpy as np
import glob

### Load Crime Data

In [4]:
# Find all crime data CSV files matching the naming pattern
csv_files = glob.glob("data/crime_split_*.csv")

# Read in all of the files and create Pandas dataframes from them
split_dfs = [pd.read_csv(f) for f in csv_files]

# Concatenate all the dataframes into a single one
df = pd.concat(split_dfs, ignore_index=True)

display(df.head())

original_dataframe_shape = df.shape
print(f'Loaded {original_dataframe_shape[0]} records!')

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
0,20000,2020467360,2020467360299901,2999,1,criminal-mischief-mtr-veh,public-disorder,8/2/2020 10:43:00 PM,NaN,8/2/2020 10:43:00 PM,...,-105.024597,39.689751,4,422,ruby-hill,1,0,1,3.133789e+06,1.676471e+06
1,20001,20196003434,20196003434299901,2999,1,criminal-mischief-mtr-veh,public-disorder,4/20/2019 7:30:00 AM,4/20/2019 8:45:00 AM,4/20/2019 6:30:00 PM,...,-104.987348,39.714316,3,311,speer,1,0,1,3.144221e+06,1.685476e+06
2,20002,2020123049,2020123049299901,2999,1,criminal-mischief-mtr-veh,public-disorder,2/25/2020 11:30:00 PM,2/26/2020 6:45:00 AM,2/26/2020 7:30:00 AM,...,-104.988098,39.764528,6,612,five-points,1,0,1,3.143907e+06,1.703765e+06
3,20003,2021621685,2021621685299901,2999,1,criminal-mischief-mtr-veh,public-disorder,11/1/2021 6:20:00 AM,11/1/2021 6:30:00 AM,11/1/2021 10:15:00 AM,...,-105.004665,39.739669,1,123,lincoln-park,1,0,1,3.139299e+06,1.694684e+06
4,20004,2020289138,2020289138299901,2999,1,criminal-mischief-mtr-veh,public-disorder,5/9/2020 12:00:00 PM,NaN,5/11/2020 12:05:00 PM,...,-104.830661,39.795281,5,521,montbello,1,0,1,3.188083e+06,1.715255e+06


Loaded 394736 records!


### Column Analysis
The next thing to do is determine which columns will be useful for predictions and which columns need to be discarded.

In [5]:
print("Crime Dataset Columns:")
for column in df.columns:
    print(f" - {column}")

Crime Dataset Columns:
 - OBJECTID
 - INCIDENT_ID
 - OFFENSE_ID
 - OFFENSE_CODE
 - OFFENSE_CODE_EXTENSION
 - OFFENSE_TYPE_ID
 - OFFENSE_CATEGORY_ID
 - FIRST_OCCURRENCE_DATE
 - LAST_OCCURRENCE_DATE
 - REPORTED_DATE
 - INCIDENT_ADDRESS
 - GEO_X
 - GEO_Y
 - GEO_LON
 - GEO_LAT
 - DISTRICT_ID
 - PRECINCT_ID
 - NEIGHBORHOOD_ID
 - IS_CRIME
 - IS_TRAFFIC
 - VICTIM_COUNT
 - x
 - y


#### Column Decisions
| Column ID                 | Decison   | Explanation                                                                                        |
|---------------------------|-----------|----------------------------------------------------------------------------------------------------|
| `OBJECTID`                | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `INCIDENT_ID`             | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_ID`              | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE`            | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_CODE_EXTENSION`  | Discarded | Related to the database how how the data is stored, identified, or queried. No predictive value.   |
| `OFFENSE_TYPE_ID`         | Kept      | This is part of the output (y-values). We will keep this column and it will be the subject of prediction by the classification model. |
| `OFFENSE_CATEGORY_ID`     | Discarded | This is part of the output (y-values). This will be discarded in  |
| `FIRST_OCCURRENCE_DATE`   | Kept      | Timestamp of the event, this will be kept and used for predictions made based on the time of day. |
| `LAST_OCCURRENCE_DATE`    | Discarded | Undefined for many rows. We will discard this timestamp and rely on `FIRST_OCCURRENCE_DATE` instead.                              |
| `REPORTED_DATE`           | Discarded | This is part of the output (y-values). Report time delta from occurrence varies incident to incident. No predictive value.                             |
| `INCIDENT_ADDRESS`        | Discarded | Although address is useful, this column is not standardized with entries like "2400 block" rather than exact addresses in some cases. |
| `GEO_X`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_Y`                   | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |
| `GEO_LON`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `GEO_LAT`                 | Kept      | Location of the crime, this will be kept and used to make predictions based on location. |
| `DISTRICT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                    |
| `PRECINCT_ID`             | Discarded | This is part of the output (y-values) and we have no way of querying. No predictive value.                             |
| `NEIGHBORHOOD_ID`         | Discarded | Although neighborhood is useful, as some are "rougher" than others, we don't have a standard way of querying.                             |
| `IS_CRIME`                | Discarded | This is part of the output (y-values). No predictive value.                             |
| `IS_TRAFFIC`              | Discarded | This is part of the output (y-values). No predictive value.                             |
| `VICTIM_COUNT`            | Discarded | This is part of the output (y-values). No predictive value.                             |
| `x`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                     |
| `y`                       | Discarded | Related to the mapping platform the data was posted on. No predictive value.                    |

In [6]:
# Keep only the columns that we want
df = df[['OFFENSE_TYPE_ID','FIRST_OCCURRENCE_DATE','GEO_LON','GEO_LAT']]

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=20))

,OFFENSE_TYPE_ID,FIRST_OCCURRENCE_DATE,GEO_LON,GEO_LAT
27980,theft-from-bldg,4/16/2021 4:39:00 AM,-105.018823,39.748246
235673,assault-simple,10/7/2021 12:10:00 AM,-104.956290,39.735148
56713,theft-items-from-vehicle,1/14/2019 10:00:00 PM,-104.996248,39.758565
115209,violation-of-restraining-order,8/4/2022 12:00:00 PM,-104.883086,39.648501
202052,theft-of-motor-vehicle,12/11/2020 10:05:00 AM,-105.025716,39.761347
208024,drug-poss-paraphernalia,7/19/2024 9:14:00 PM,-104.978646,39.739219
350049,threats-to-injure,7/29/2023 1:30:00 PM,-104.775976,39.783554
219225,criminal-mischief-other,6/7/2024 4:11:00 PM,-104.971004,39.736072
34554,drug-methampetamine-sell,5/26/2021 1:35:00 PM,-104.987485,39.739898
70583,criminal-mischief-other,6/27/2021 9:05:00 PM,-104.900525,39.767595


### Time Binning
In this analysis, time binning is used to capture the cyclical and seasonal patterns inherent in criminal behavior. Crime often follows temporal trends—certain types of incidents may be more likely to occur at specific times of day, days of the week, or during particular seasons. For example, incidents related to nightlife might peak during late evening hours, while other crimes may correlate with daily commuter traffic or specific days like weekends. By extracting temporal features like `YEAR`, `DAY_OF_YEAR`, `DAY_OF_WEEK`, and a continuous `TIME` (floating-point hour), we enable the model to identify these patterns and trends. This granular binning enhances the predictive power of the model, allowing it to detect not only broad time-based patterns but also more subtle nuances in how crime evolves throughout the day, week, and year.

#### Temporal Features Breakdown
* `YEAR`: Provides a clear yearly context, which can be crucial for identifying long-term trends or annual shifts in patterns.
* `TIME` (floating point): Offers a precise measure of the time of day, allowing models to pick up on subtle temporal shifts within a 24-hour cycle.
* `DAY_OF_YEAR`: Captures the day within the year, useful for identifying seasonal patterns or trends that recur annually.
* `DAY_OF_WEEK`: Encodes weekly cycles, helping the model detect patterns associated with specific weekdays.

In [7]:
# Convert FIRST_OCCURRENCE_DATE to Pandas date time format
df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(df['FIRST_OCCURRENCE_DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date and time components
df['YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.year
# df['MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.month -- Removed in favor of DAY_OF_YEAR
# df['DAY_OF_MONTH'] = df['FIRST_OCCURRENCE_DATE'].dt.day -- Removed in favor of DAY_OF_YEAR
# df['HOUR'] = df['FIRST_OCCURRENCE_DATE'].dt.hour -- Removed in favor of a floating point hour describing the time of day with more precision
df['TIME'] = (
    df['FIRST_OCCURRENCE_DATE'].dt.hour + 
    df['FIRST_OCCURRENCE_DATE'].dt.minute / 60 + 
    df['FIRST_OCCURRENCE_DATE'].dt.second / 3600
)
df['DAY_OF_YEAR'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofyear
df['DAY_OF_WEEK'] = df['FIRST_OCCURRENCE_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday

# Finally, dropping the FIRST_OCCURRENCE_DATE column
df.drop(columns=['FIRST_OCCURRENCE_DATE'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

,OFFENSE_TYPE_ID,GEO_LON,GEO_LAT,YEAR,TIME,DAY_OF_YEAR,DAY_OF_WEEK
245688,assault-simple,-104.905633,39.677515,2024,3.066667,28,6
234815,assault-simple,-104.965344,39.742402,2021,17.750000,291,0
321468,theft-other,-104.775920,39.807865,2021,17.000000,251,2
343944,weapon-unlawful-discharge-of,-105.007685,39.765869,2022,19.333333,294,4
347395,theft-parts-from-vehicle,-104.929727,39.727898,2023,14.750000,198,0


### Location Slicing & Binning 
In this step, we convert latitude and longitude into a 15-meter by 15-meter grid system using a projected coordinate system (UTM). This method ensures that each crime location is assigned to a unique, precise grid cell, facilitating accurate spatial analysis.

See <https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system>

#### Why This is Better Than Using Raw Latitude and Longitude

Using UTM coordinates overcomes the limitations of raw latitude and longitude, which vary in distance depending on location. UTM coordinates provide uniform distance measurements in meters, improving precision and making it easier to define consistent grid cells. This approach also simplifies spatial analysis by grouping data into fixed-size cells, making patterns like crime hotspots easier to identify and computationally more efficient to process.

In [8]:
GRID_SIZE = 15 # 15m x 15m grid cells

# Initialize UTM projection (assuming UTM Zone 13N for Denver)
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system
#   https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")

def get_grid_block(lat, lon):
    try:
        # Convert latitude and longitude to UTM coordinates (X, Y)
        x, y = utm_proj(lon, lat)  # Correct order: lon, lat
        
        # Calculate the X and Y blocks
        x_block = int(x // GRID_SIZE)
        y_block = int(y // GRID_SIZE)
        
        return x_block, y_block
    except:
        # If there is an issue parsing, return -1
        return -1, -1

# Apply the function to the DataFrame
df[['X_BLOCK', 'Y_BLOCK']] = df.apply(lambda row: get_grid_block(row['GEO_LAT'], row['GEO_LON']), axis=1, result_type='expand')

# Drop rows where X_BLOCK or Y_BLOCK is -1
df = df[(df['X_BLOCK'] != -1) & (df['Y_BLOCK'] != -1)]

# Note change in size
post_null_geo_block_removal_dataframe_shape = df.shape
print(f'Dataframe size before removal: {original_dataframe_shape[0]} items')
print(f'Dataframe size after removal: {post_null_geo_block_removal_dataframe_shape[0]} items')
print(f'--> {original_dataframe_shape[0] - post_null_geo_block_removal_dataframe_shape[0]} items removed!')

# Finally, dropping the GEO_LAT and GEO_LON columns since we have replaced them with X_BLOCK and Y_BLOCK
df.drop(columns=['GEO_LAT'], inplace=True)
df.drop(columns=['GEO_LON'], inplace=True)

# Displaying a random sample of the data to confirm what it looks like
display(df.sample(n=5))

Dataframe size before removal: 394736 items
Dataframe size after removal: 394475 items
--> 261 items removed!


,OFFENSE_TYPE_ID,YEAR,TIME,DAY_OF_YEAR,DAY_OF_WEEK,X_BLOCK,Y_BLOCK
185691,curfew,2020,0.750000,154,1,33414,293229
136664,weapon-unlawful-discharge-of,2021,20.166667,298,0,33805,293439
72356,criminal-mischief-other,2020,0.766667,157,4,33367,293496
322837,theft-bicycle,2020,8.000000,316,2,33096,293285
270098,theft-of-motor-vehicle,2020,17.000000,7,1,33182,292864


In this section, we are processing the geographical data to assign each crime incident to a specific grid cell based on its latitude and longitude. We achieve this by converting the coordinates to UTM projection, which provides more accurate distance-based calculations than traditional latitude and longitude. For each point, we calculate the corresponding grid cell by dividing the UTM coordinates by the grid size (15 meters) and then storing the results as `X_BLOCK` and `Y_BLOCK`.

However, if any coordinates cannot be processed correctly (due to projection issues or invalid data), we assign -1 as a placeholder. After applying this transformation, we remove any rows where either the `X_BLOCK` or `Y_BLOCK` is -1, indicating invalid grid assignments. The sizes of the DataFrame before and after this removal are printed to highlight the impact of cleaning the data. This step ensures that only valid data points are included for further analysis and modeling.

### Removing Location Outliers
When exporting the data as a CSV file, it can be observed that some of the rows have really low or high X-block values. These are likely location mis-inputs.

We can drop these rows by calculating the interquartile range range for the `X_BLOCK` and `Y_BLOCK` columns and dropping the rows that are outliers.

What I have done below is not the interquartile range. I only want to drop extreme outliers, so I dropped the values that fall outside the middle 98% of rows.

In [9]:
df_shape_before_location_outlier_drop = df.shape

Q1 = df['X_BLOCK'].quantile(0.01)
Q3 = df['X_BLOCK'].quantile(0.99)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df.loc[(df['X_BLOCK'] >= lower_bound) & (df['X_BLOCK'] <= upper_bound)]

# Calculate and display rows removed
df_shape_after_location_outlier_drop = df.shape
print(f'Size before location outlier removal: {df_shape_before_location_outlier_drop[0]} records.')
print(f'Size after location outlier removal: {df_shape_after_location_outlier_drop[0]} records.')
print(f'--> {df_shape_before_location_outlier_drop[0] - df_shape_after_location_outlier_drop[0]} records removed!')


Size before location outlier removal: 394475 records.
Size after location outlier removal: 394434 records.
--> 41 records removed!


### Adding in OpenStreetMap Features
In this section, we aim to enrich our dataset with spatial features from OpenStreetMap (OSM). By querying OSM data for geographic features based on the centroids of the X and Y grid blocks (calculated from latitude and longitude), we can incorporate valuable context such as the proximity to roads, points of interest, or land use types. This spatial data enhances our analysis by providing a richer understanding of the environment in which the crimes occur, potentially improving predictive modeling. By leveraging OpenStreetMap data, we can gather additional information about the geographical characteristics of each grid cell, allowing for more nuanced predictions based on spatial context.

In [15]:
# Imports
import sqlite3
from tqdm import tqdm
import re
# Import osmium to parse the OSM files
import osmium

# Re-define the UTM projections from earlier
utm_proj = pyproj.Proj(proj="utm", zone=13, datum="WGS84")
latlon_proj = pyproj.Proj(proj="latlong", datum="WGS84")

# Define tranformers
latlon_to_utm_transformer = pyproj.Transformer.from_proj(latlon_proj, utm_proj)
utm_to_latlon_transformer = pyproj.Transformer.from_proj(utm_proj, latlon_proj)

'''
    Generate a Grid of Map Blocks/Cells from predefined corners.
'''
def gen_blocks_from_coordinates(southwest_corner, northeast_corner, database_path):
    # Bounding corners
    def dec_coordinates(str_coordinates):
        parts = str_coordinates.split(",")
        
        # Extract latitude from the first part and longitude from the second part
        lat = float(re.findall(r"\d+\.\d+", parts[0])[0])
        lon = float(re.findall(r"\d+\.\d+", parts[1])[0])
        
        # Adjust sign based on N/S and E/W indicators
        if "S" in parts[0]:
            lat = -lat
        if "W" in parts[1]:
            lon = -lon
            
        return lon, lat  # Return (longitude, latitude)

    southwest_lon, southwest_lat = dec_coordinates(southwest_corner)
    northeast_lon, northeast_lat = dec_coordinates(northeast_corner)

    # Define block size in meters (15 meters as mentioned)
    block_size = 15

    # Create a transformer for converting lat/lon to UTM
    latlon_to_utm_transformer = pyproj.Transformer.from_proj(latlon_proj, utm_proj)

    # Convert southwest and northeast corners to UTM
    southwest_x, southwest_y = latlon_to_utm_transformer.transform(southwest_lon, southwest_lat)
    northeast_x, northeast_y = latlon_to_utm_transformer.transform(northeast_lon, northeast_lat)

    # Calculate the area width and height in meters
    area_width = northeast_x - southwest_x
    area_height = northeast_y - southwest_y

    # Calculate the number of blocks in the area
    num_x_blocks = int(area_width // block_size)
    num_y_blocks = int(area_height // block_size)

    # Create SQLite database and table
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create a table to store the map cells
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS blocks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            x_block INTEGER,
            y_block INTEGER,
            cent_lat REAL,
            cent_lon REAL
        )
    ''')

    # Function to calculate the centroid of a block (in UTM)
    def get_centroid(x_block, y_block, grid_size=15):
        # Create a transformer for converting UTM to lat/lon
        utm_to_latlon_transformer = pyproj.Transformer.from_proj(utm_proj, latlon_proj)
        
        # Get the bottom-left corner coordinates in UTM
        x_origin, y_origin = x_block * grid_size, y_block * grid_size
        
        # Calculate the centroid by moving half the grid size in both directions
        x_centroid = x_origin + grid_size / 2
        y_centroid = y_origin + grid_size / 2
        
        # Convert UTM coordinates to latitude and longitude
        lon, lat = utm_to_latlon_transformer.transform(x_centroid, y_centroid)
        
        return lat, lon

    # Initialize the progress bar
    total_blocks = num_x_blocks * num_y_blocks

    # Set the batch size
    batch_size = 100000
    batch_data = []

    # Init the progress bar
    progress_bar = tqdm(total=total_blocks, desc="Identifying Block Centroids", position=0, ncols=120)

    # Loop through each block and calculate the centroid
    for y_block in range(num_y_blocks):
        for x_block in range(num_x_blocks):
            # Calculate the centroid for the current block
            lat, lon = get_centroid(x_block, y_block, block_size)
            
            # Append data to batch
            batch_data.append((x_block, y_block, lat, lon))
            
            # If batch size is reached, execute the batch insert
            if len(batch_data) >= batch_size:
                cursor.executemany('''
                    INSERT INTO blocks (x_block, y_block, cent_lat, cent_lon)
                    VALUES (?, ?, ?, ?)
                ''', batch_data)
                
                # Manually update the progress bar after each block
                progress_bar.update(len(batch_data))
                
                conn.commit()  # Commit after each batch
                batch_data = []  # Clear the batch data

    # Insert any remaining data after the loop
    if batch_data:
        cursor.executemany('''
            INSERT INTO blocks (x_block, y_block, cent_lat, cent_lon)
            VALUES (?, ?, ?, ?)
        ''', batch_data)

        progress_bar.update(len(batch_data))

        conn.commit()  # Final commit for remaining data

    # Close the progress bar once finished
    progress_bar.close()

    # Close the SQLite connection
    conn.close()

'''
    Generate a map based on the boundaries of the available crime location data, plus a margin
'''
def gen_map_from_crime_location_data_with_margin(crime_df, margin, csv_path):
    # Get the limits of the locations saved within the data frame
    x_max = crime_df['X_BLOCK'].max()
    x_min = crime_df['X_BLOCK'].min()
    y_max = crime_df['Y_BLOCK'].max()
    y_min = crime_df['Y_BLOCK'].min()

    # Calculate the limits with the 
    x_max_margin = x_max + margin
    x_min_margin = x_min - margin
    y_max_margin = y_max + margin
    y_min_margin = y_min - margin

    # Print the map size
    print(f"Map Size Before Margin (in blocks): Upper Left Corner: ({x_min}, {y_max}), Lower Right Corner: ({x_max}, {y_min})")
    print(f"Map Size After Margin (in blocks): Upper Left Corner: ({x_min_margin}, {y_max_margin}), Lower Right Corner: ({x_max_margin}, {y_min_margin})")

    # Function to get the edges of the box in lon/lat
    def block_to_latlon(x, y):
        # Project from UTM to latlon
        transformer = pyproj.Transformer.from_crs(f"epsg:32613", "epsg:4326", always_xy=True)
        return transformer.transform(x * GRID_SIZE, y * GRID_SIZE)

    min_longitude, min_latitude = block_to_latlon(x_min_margin, y_min_margin)
    max_longitude, max_latitude = block_to_latlon(x_max_margin, y_max_margin)
    print(f"    ({min_longitude},{min_latitude}) ({max_longitude},{max_latitude})")

    # Loop through the each possible block in the map, including the margins
    print(f'--> Total map grid cells: {(x_max_margin - x_min_margin) * (y_max_margin - y_min_margin)}')
    map_cells = []
    for x in range(x_min_margin, x_max_margin + 1):
        for y in range(y_min_margin, y_max_margin + 1):
            # Get important latitudes and longitudes
            max_lon, min_lat = block_to_latlon(x, y)
            min_lon, max_lat = block_to_latlon(x + 1, y - 1)
            center_lon, center_lat = block_to_latlon(x + (GRID_SIZE / 2), y - (GRID_SIZE / 2))

            for obj in osmium.FileProcessor('../denver_filtered.osm.pbf', osmium.osm.NODE):
                if osmium.geom.haversine_distance(osmium.osm.Location(center_lon, center_lat), obj.location) < GRID_SIZE:
                    print(obj)

            map_cells.append({
                'x': x,
                'y': y,
                'c_lon': center_lon,
                'c_lat': center_lat,
                'land_use': obj
            })

# Old method was just a fixed area
southwest_corner = "39.53440° N, 105.24364° W"
northeast_corner = "40.03841° N, 104.58877° W"
database_path = 'map_grid.db'
# gen_blocks_from_coordinates(southwest_corner, northeast_corner, database_path)

gen_map_from_crime_location_data_with_margin(df, 400, 'map.csv')

Map Size Before Margin (in blocks): Upper Left Corner: (32154, 294550), Lower Right Corner: (35553, 292331)
Map Size After Margin (in blocks): Upper Left Corner: (31754, 294950), Lower Right Corner: (35953, 291931)
    (-105.2757695961267,39.560047846577135) (-104.53987298311186,39.96748737870319)
--> Total map grid cells: 12676781


KeyboardInterrupt: 